# Read Raw

In [1]:
import pandas as pd

# Replace 'your_file.csv.gz' with the path to your .csv.gz file
df = pd.read_csv('/home/ubuntu/Rheza/local-share/06_trades_and_orderbooks/BTCUSDT2025-01-11.csv.gz', compression='gzip')

# Convert the 'timestamp' column to integers
df['timestamp'] = df['timestamp'] * 1000
df['timestamp'] = df['timestamp'].astype(int)

# Drop all columns after 'price'
df = df[['timestamp','side','size','price', 'tickDirection']]

# Display the first few rows of the dataframe
df

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/Rheza/local-share/06_trades_and_orderbooks/BTCUSDT2025-01-11.csv.gz'

In [23]:
# df.to_csv('cleaned_trades_BTCUSDT2025-01-11.csv', index=False)

# Read Simplified Data

In [2]:
import pandas as pd

# Replace 'your_file.csv.gz' with the path to your .csv.gz file
df = pd.read_csv('/home/ubuntu/Rheza/local-share/06_trades_and_orderbooks/cleaned_trades_BTCUSDT2025-01-11.csv')

df

,timestamp,side,size,price,tickDirection
0,1736553600326,Buy,0.001,94684.3,PlusTick
1,1736553600781,Sell,0.082,94684.2,MinusTick
2,1736553600831,Sell,0.001,94684.2,ZeroMinusTick
3,1736553601018,Sell,0.001,94684.2,ZeroMinusTick
4,1736553601022,Sell,0.002,94684.2,ZeroMinusTick
...,...,...,...,...,...
677407,1736639994670,Sell,0.100,94569.4,MinusTick
677408,1736639997157,Sell,0.010,94569.4,ZeroMinusTick
677409,1736639998097,Sell,0.001,94569.4,ZeroMinusTick
677410,1736639998421,Sell,0.001,94569.4,ZeroMinusTick


# Data Cleaning

In [3]:
df

,timestamp,side,size,price,tickDirection
0,1736553600326,Buy,0.001,94684.3,PlusTick
1,1736553600781,Sell,0.082,94684.2,MinusTick
2,1736553600831,Sell,0.001,94684.2,ZeroMinusTick
3,1736553601018,Sell,0.001,94684.2,ZeroMinusTick
4,1736553601022,Sell,0.002,94684.2,ZeroMinusTick
...,...,...,...,...,...
677407,1736639994670,Sell,0.100,94569.4,MinusTick
677408,1736639997157,Sell,0.010,94569.4,ZeroMinusTick
677409,1736639998097,Sell,0.001,94569.4,ZeroMinusTick
677410,1736639998421,Sell,0.001,94569.4,ZeroMinusTick


In [2]:
import numpy as np

# Check for NaN values
nan_counts = df.isna().sum()

# Check for infinite values
inf_counts = np.isinf(df[['price', 'size']]).sum()

# Check for zero values in 'price' and 'size'
zero_counts = (df[['price', 'size']] == 0).sum()

# Display results
print("NaN counts:\n", nan_counts)
print("\nInf counts:\n", inf_counts)
print("\nZero counts:\n", zero_counts)

NaN counts:
 timestamp        0
side             0
size             0
price            0
tickDirection    0
dtype: int64

Inf counts:
 price    0
size     0
dtype: int64

Zero counts:
 price    0
size     0
dtype: int64


# Create OHLC

In [18]:
import pandas as pd

def aggregate_ohlc(df, timestamp_col='timestamp', price_col='price', interval='1s'):
    """
    Aggregates trade data into OHLC bars over a specified time interval,
    calculates max absolute percentage change, and determines the percentage 
    of rows where max_abs_change >= 0.1% and >= 0.07%.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame containing at least a timestamp and price columns.
        timestamp_col (str): Name of the column with the Unix timestamp (in ms). Default is 'timestamp'.
        price_col (str): Name of the price column. Default is 'price'.
        interval (str): Resampling interval (e.g., '1s' for 1 second, '5min' for 5 minutes, '1h' for 1 hour, '2h' for 2 hours).
    
    Returns:
        tuple: A tuple containing:
            - pd.DataFrame: DataFrame with a datetime column ('time_bin'), OHLC columns, and max_abs_change.
            - int: Total number of rows in the OHLC DataFrame that have at least one NaN in the OHLC columns.
            - float: Percentage of rows where max_abs_change >= 0.1.
            - float: Percentage of rows where max_abs_change >= 0.07.
    """
    # Convert the Unix timestamp (in ms) to datetime
    df['datetime'] = pd.to_datetime(df[timestamp_col], unit='ms')
    
    # Create a new column by flooring the datetime to the nearest specified interval
    df['time_bin'] = df['datetime'].dt.floor(interval)
    
    # Group by the time_bin column and calculate OHLC for the price column
    ohlc = df.groupby('time_bin')[price_col].agg(
        open='first',
        high='max',
        low='min',
        close='last'
    )
    
    # Determine the full range of time bins using the min and max timestamps from the data
    start = df['datetime'].min().floor(interval)
    end = df['datetime'].max().ceil(interval)
    full_range = pd.date_range(start=start, end=end, freq=interval)
    
    # Reindex the aggregated OHLC data to include every time bin in the computed range,
    # filling missing intervals with NaN values
    ohlc = ohlc.reindex(full_range)
    
    # Reset the index so that 'time_bin' becomes a column
    ohlc = ohlc.reset_index().rename(columns={'index': 'time_bin'})

    # Calculate max_abs_change
    def calculate_max_abs_change(row):
        if pd.isna(row['open']) or pd.isna(row['high']) or pd.isna(row['low']):
            return None
        change_high = (row['high'] - row['open']) / row['open'] * 100 if row['high'] != row['open'] else 0
        change_low = (row['open'] - row['low']) / row['open'] * 100 if row['low'] != row['open'] else 0
        return max(change_high, change_low)

    ohlc['max_abs_change'] = ohlc.apply(calculate_max_abs_change, axis=1)

    # Count the total number of rows that have at least one NaN value in the OHLC columns
    nan_rows_total = ohlc[['open', 'high', 'low', 'close']].isna().any(axis=1).sum()

    # Calculate the percentage of rows where max_abs_change >= 0.1 and >= 0.07
    valid_rows = ohlc['max_abs_change'].notna().sum()
    maker_maker_count = (ohlc['max_abs_change'] >= 0.1).sum()
    maker_taker_count = (ohlc['max_abs_change'] >= 0.07).sum()

    percentage_maker_maker = (maker_maker_count / valid_rows) * 100 if valid_rows > 0 else 0
    percentage_maker_taker = (maker_taker_count / valid_rows) * 100 if valid_rows > 0 else 0

    return ohlc, nan_rows_total, percentage_maker_maker, percentage_maker_taker

In [23]:
# List of intervals
intervals = ['1s', '5s','15s','30s', '1min', '2min','3min','4min', '5min', '6min', '7min', '8min', '9min', '10min']

# Store results in a DataFrame
results = []
for interval in intervals:
    _, nan_rows_count, maker_maker_pct, maker_taker_pct = aggregate_ohlc(df, interval=interval)
    results.append([interval, nan_rows_count, maker_maker_pct, maker_taker_pct])

# Create a DataFrame to display results
summary_df = pd.DataFrame(results, columns=['Interval', 'Total NaN Rows', 'Maker - Maker % (>=0.1)', 'Maker - Taker % (>=0.07)'])
summary_df

,Interval,Total NaN Rows,Maker - Maker % (>=0.1),Maker - Taker % (>=0.07)
0,1s,23464,0.001589,0.007945
1,5s,108,0.023292,0.075700
2,15s,1,0.156250,0.468750
3,30s,1,0.451389,1.840278
4,1min,1,1.944444,6.944444
5,2min,1,6.527778,17.916667
6,3min,1,15.416667,33.125000
7,4min,1,18.333333,43.055556
8,5min,1,24.305556,50.000000
9,6min,1,31.250000,57.083333


# Feature Engineering

In [9]:
df_features = df.copy()

df_features['quantity'] = df_features['size'] * df_features['price']
df_features

,side,size,price,tickDirection,quantity
timestamp,,,,,
2025-01-11 00:00:00.326,Buy,0.001,94684.3,PlusTick,94.6843
2025-01-11 00:00:00.781,Sell,0.082,94684.2,MinusTick,7764.1044
2025-01-11 00:00:00.831,Sell,0.001,94684.2,ZeroMinusTick,94.6842
2025-01-11 00:00:01.018,Sell,0.001,94684.2,ZeroMinusTick,94.6842
2025-01-11 00:00:01.022,Sell,0.002,94684.2,ZeroMinusTick,189.3684
...,...,...,...,...,...
2025-01-11 23:59:54.670,Sell,0.100,94569.4,MinusTick,9456.9400
2025-01-11 23:59:57.157,Sell,0.010,94569.4,ZeroMinusTick,945.6940
2025-01-11 23:59:58.097,Sell,0.001,94569.4,ZeroMinusTick,94.5694


In [3]:
print(f'Min Timestamp: {df['timestamp'].min()}')
print(f'Max Timestamp: {df['timestamp'].max()}')

Min Timestamp: 1736553600326
Max Timestamp: 1736639998908


In [8]:
import pandas as pd

# Assuming your DataFrame is named `df`
# Convert the timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Set the timestamp as the index
df.set_index('timestamp', inplace=True)

# Resample the data per second and calculate OHLC
ohlc_df = df['price'].resample('1S').agg(
    Open=('first'),
    High=('max'),
    Low=('min'),
    Close=('last')
)

# Fill missing seconds with NaN (if there are any)
ohlc_df = ohlc_df.asfreq('1M')

# Reset the index to make timestamp a column again (optional)
ohlc_df.reset_index(inplace=True)

# Check the shape of the resulting DataFrame
print(ohlc_df.shape)  # Should be (86400, 5)

# Display the first few rows
print(ohlc_df.head())

KeyError: 'timestamp'

In [5]:
ohlc_df

,timestamp,Open,High,Low,Close
0,2025-01-11 00:00:00,94684.3,94684.3,94684.2,94684.2
1,2025-01-11 00:00:01,94684.2,94684.3,94684.2,94684.2
2,2025-01-11 00:00:02,94684.3,94684.3,94684.2,94684.2
3,2025-01-11 00:00:03,94684.3,94684.3,94684.2,94684.2
4,2025-01-11 00:00:04,94684.2,94684.3,94684.2,94684.3
...,...,...,...,...,...
86394,2025-01-11 23:59:54,94569.4,94569.4,94569.4,94569.4
86395,2025-01-11 23:59:55,NaN,NaN,NaN,NaN
86396,2025-01-11 23:59:56,NaN,NaN,NaN,NaN
86397,2025-01-11 23:59:57,94569.4,94569.4,94569.4,94569.4


In [6]:
import pandas as pd
import numpy as np

# Ensure df_features is sorted by timestamp
df_features = df_features.sort_values(by="timestamp")

# Define the target timestamp bins (in milliseconds)
target_ts_list = list(range(1738022405000, 1738108805000, 5000))

# Initialize an empty list to store the results
results = []

for target_ts in target_ts_list:
    start_ts = target_ts - 5000  # Define the start of the window
    end_ts = target_ts  # Define the end of the window
    
    # Filter data within the time window
    df_window = df_features[(df_features["timestamp"] >= start_ts) & (df_features["timestamp"] < end_ts)]
    
    # Compute sum of buy and sell quoteprice
    sum_buy_quoteprice = df_window.loc[df_window["side"] == "Buy", "quoteprice"].sum()
    sum_sell_quoteprice = df_window.loc[df_window["side"] == "Sell", "quoteprice"].sum()
    
    # Compute sum of buy and sell size
    sum_buy_size = df_window.loc[df_window["side"] == "Buy", "size"].sum()
    sum_sell_size = df_window.loc[df_window["side"] == "Sell", "size"].sum()
    
    # Nearest price (i.e., the price of the row with the closest timestamp to the target timestamp)
    if not df_window.empty:
        nearest_idx = (df_window["timestamp"] - target_ts).abs().idxmin()
        nearest_price = df_window.loc[nearest_idx, "price"]
    else:
        nearest_price = np.nan
    
    # Buy price with the most sum of size
    if not df_window[df_window["side"] == "Buy"].empty:
        buy_price_most_size = df_window[df_window["side"] == "Buy"].groupby("price")["size"].sum().idxmax()
    else:
        buy_price_most_size = np.nan
    
    # Sell price with the most sum of size
    if not df_window[df_window["side"] == "Sell"].empty:
        sell_price_most_size = df_window[df_window["side"] == "Sell"].groupby("price")["size"].sum().idxmax()
    else:
        sell_price_most_size = np.nan

    # Append results to list
    results.append([
        target_ts, sum_buy_quoteprice, sum_sell_quoteprice, sum_buy_size, sum_sell_size,
        nearest_price, buy_price_most_size, sell_price_most_size
    ])

# Create DataFrame from results
df_summary = pd.DataFrame(results, columns=[
    "timestamp", "sum_buy_quoteprice", "sum_sell_quoteprice", "sum_buy_size", "sum_sell_size",
    "nearest_price", "buy_price_most_size", "sell_price_most_size"
])

# Display summary
df_summary

,timestamp,sum_buy_quoteprice,sum_sell_quoteprice,sum_buy_size,sum_sell_size,nearest_price,buy_price_most_size,sell_price_most_size
0,1738022405000,10816.0120,89359.0653,3.40,28.09,3181.18,3181.18,3181.17
1,1738022410000,207855.1386,149413.4592,65.34,46.97,3180.59,3181.18,3181.17
2,1738022415000,85201.5679,44175.9577,26.79,13.89,3180.73,3180.27,3180.37
3,1738022420000,67215.3038,53759.6228,21.13,16.90,3180.84,3181.18,3181.10
4,1738022425000,83908.5168,11164.6134,26.38,3.51,3181.17,3180.79,3180.84
...,...,...,...,...,...,...,...,...
17275,1738108780000,5507.1191,92.2959,1.79,0.03,3076.93,3076.54,3076.53
17276,1738108785000,4615.6650,5354.1540,1.50,1.74,3077.10,3077.11,3077.10
17277,1738108790000,2984.0676,104453.1112,0.97,33.95,3075.89,3076.62,3077.10
17278,1738108795000,8488.3328,6059.1430,2.76,1.97,3076.19,3075.38,3075.75


In [7]:
df_summary.to_csv('cleaned_trades_20250128.csv', index=False)

# Binance Agg Trades

In [37]:
# List of Parquet file paths
file_paths_btc = [
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-01_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-02_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-03_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-04_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-05_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-06_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-07_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-08_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-09_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-10_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-11_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_btcusdt/aggtrades_data_5_sec/BTC_USDT_AGGTRADES_PERPS_2024-12_5_sec.parquet"
]

In [94]:
# List of Parquet file paths
file_paths_eth = [
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-01_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-02_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-03_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-04_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-05_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-06_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-07_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-08_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-09_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-10_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-11_5_sec.parquet",
    "/home/ubuntu/trades_data/binance/perps_ethusdt/aggtrades_data_5_sec/ETH_USDT_AGGTRADES_PERPS_2024-12_5_sec.parquet"
]

In [95]:
# Function to read data from multiple files (combined)
def read_data_from_files(file_paths):
    dfs = []
    for path in file_paths:
        df = pl.read_parquet(path)
        dfs.append(df)
    return pl.concat(dfs)

dfs = read_data_from_files(file_paths_eth)
dfs

timestamp_5_second,date,year_month,hourminute,openprice,highprice,lowprice,closeprice,std_price,sum_quotevolume,sum_trades,avg_quotevolume_per_trade,buyer_maker_price_wa,buyer_taker_price_wa,wa_price,buyer_maker_price_std,buyer_maker_quotevolume,buyer_maker_trades,buyer_taker_price_std,buyer_taker_quotevolume,buyer_taker_trades
str,str,str,str,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,i64
"""2024-01-0100:00:00""","""2024-01-01""","""2024-01""","""00:00""",2283.84,2284.16,2283.77,2284.15,0.14558,415581.27552,147,2253.306862,2283.834082,2283.822561,2283.827704,0.158639,185534.11298,55,0.139688,230047.16254,92
"""2024-01-0100:00:05""","""2024-01-01""","""2024-01""","""00:00""",2284.15,2284.15,2283.29,2283.32,0.270035,624944.23524,316,1760.548754,2283.523824,2283.395361,2283.518632,0.269842,599685.31592,283,0.215192,25258.91932,33
"""2024-01-0100:00:10""","""2024-01-01""","""2024-01""","""00:00""",2283.31,2283.39,2282.97,2283.39,0.123572,665066.77796,254,1798.921204,2283.243631,2283.264709,2283.248063,0.119339,525221.38127,143,0.127461,139845.39669,111
"""2024-01-0100:00:15""","""2024-01-01""","""2024-01""","""00:00""",2283.38,2283.39,2283.38,2283.38,0.004423,39906.67431,38,1086.138539,2283.38,2283.39,2283.382406,0.0,30305.01936,29,0.0,9601.65495,9
"""2024-01-0100:00:20""","""2024-01-01""","""2024-01""","""00:00""",2283.38,2283.38,2283.18,2283.19,0.083084,110730.08417,39,1474.432198,2283.339996,2283.19,2283.329917,0.094516,103289.16796,31,0.0,7440.91621,8
"""2024-01-0100:00:25""","""2024-01-01""","""2024-01""","""00:00""",2283.18,2283.46,2283.18,2283.46,0.099933,183021.55763,79,2700.301925,2283.283054,2283.258199,2283.260035,0.119059,13514.75235,18,0.094092,169506.80528,61
"""2024-01-0100:00:30""","""2024-01-01""","""2024-01""","""00:00""",2283.46,2284.79,2283.46,2284.78,0.363731,502383.9759,311,1404.406094,2284.674824,2284.108729,2284.16577,0.414758,50621.53936,22,0.359943,451762.43654,289
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2024-12-3123:59:30""","""2024-12-31""","""2024-12""","""23:59""",3336.58,3336.58,3336.57,3336.57,0.005,1561.5187,13,167.941128,3336.57,3336.58,3336.578419,0.0,246.90618,10,0.0,1314.61252,3


In [96]:
import polars as pl

# Function to calculate max_abs_change and percentage of rows exceeding thresholds
def calculate_max_abs_change(df, thresholds=[0.04, 0.07, 0.1]):
    # Calculate percentage change for high-open and open-low
    df = df.with_columns([
        ((pl.col("highprice") - pl.col("openprice")) / pl.col("openprice") * 100)
        .fill_nan(0)
        .alias("perc_high_open"),
        
        ((pl.col("openprice") - pl.col("lowprice")) / pl.col("openprice") * 100)
        .fill_nan(0)
        .alias("perc_open_low")
    ])

    # Calculate max_abs_change as the maximum of perc_high_open and perc_open_low
    df = df.with_columns([
        pl.max_horizontal(["perc_high_open", "perc_open_low"]).alias("max_abs_change")
    ])

    # Calculate percentage of rows exceeding the thresholds
    total_rows = df.height
    threshold_results = {
        f"threshold_{threshold}_percent": (df.filter(pl.col("max_abs_change") >= threshold).height / total_rows) * 100
        for threshold in thresholds
    }

    return threshold_results

# Call the function
summary = calculate_max_abs_change(dfs, thresholds=[0.04, 0.07, 0.1])

# Print the results
print("\nSummary of Threshold Percentages:", summary)


Summary of Threshold Percentages: {'threshold_0.04_percent': 11.736998480515926, 'threshold_0.07_percent': 3.186504691677879, 'threshold_0.1_percent': 1.1928795986373506}


In [97]:
import polars as pl

def aggregate_to_1min(df: pl.DataFrame) -> pl.DataFrame:
    """
    Aggregates the 5-second data into 1-minute intervals and redefines the OHLC values.

    Parameters:
        df (pl.DataFrame): The input DataFrame with 5-second data.

    Returns:
        pl.DataFrame: Aggregated DataFrame with 1-minute OHLC values.
    """
    # Group by 'date', 'year_month', and 'hourminute' and aggregate OHLC
    aggregated_df = df.group_by(["date", "year_month", "hourminute"]).agg([
        pl.last("timestamp_5_second").alias("timestamp"),
        pl.first("openprice").alias("open"),  # First open price in the minute
        pl.max("highprice").alias("high"),    # Highest high price in the minute
        pl.min("lowprice").alias("low"),      # Lowest low price in the minute
        pl.last("closeprice").alias("close")  # Last close price in the minute
    ])
    
    # Sort the DataFrame by 'timestamp' in ascending order
    aggregated_df = aggregated_df.sort("timestamp")
    
    return aggregated_df

# Example usage:
aggregated_df = aggregate_to_1min(dfs)  # Assuming `dfs` is your original data
aggregated_df

date,year_month,hourminute,timestamp,open,high,low,close
str,str,str,str,f64,f64,f64,f64
"""2024-01-01""","""2024-01""","""00:00""","""2024-01-0100:00:55""",2283.84,2284.79,2282.97,2283.89
"""2024-01-01""","""2024-01""","""00:01""","""2024-01-0100:01:55""",2283.89,2285.9,2283.85,2285.68
"""2024-01-01""","""2024-01""","""00:02""","""2024-01-0100:02:55""",2285.68,2286.37,2285.27,2286.3
"""2024-01-01""","""2024-01""","""00:03""","""2024-01-0100:03:55""",2286.3,2288.44,2286.29,2288.35
"""2024-01-01""","""2024-01""","""00:04""","""2024-01-0100:04:55""",2288.36,2289.92,2288.36,2289.92
"""2024-01-01""","""2024-01""","""00:05""","""2024-01-0100:05:55""",2289.92,2291.93,2288.77,2291.45
"""2024-01-01""","""2024-01""","""00:06""","""2024-01-0100:06:55""",2291.45,2293.22,2291.45,2292.3
…,…,…,…,…,…,…,…
"""2024-12-31""","""2024-12""","""23:54""","""2024-12-3123:54:55""",3340.52,3341.09,3339.38,3339.51


In [98]:
# Call the function
summary_1m = calculate_max_abs_change(aggregated_df, thresholds=[0.04, 0.07, 0.1])

# Print the results
print("\nSummary of Threshold Percentages:", summary_1m)

ColumnNotFoundError: highprice

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'with_columns' <---
DF ["date", "year_month", "hourminute", "timestamp"]; PROJECT */8 COLUMNS; SELECTION: None

In [99]:
import polars as pl

def calculate_max_abs_change_by_year_month(df, thresholds=[0.04, 0.07, 0.1]):
    """
    Calculate max absolute change and percentage of rows exceeding thresholds for each year-month.
    
    Parameters:
        df (pl.DataFrame): The input DataFrame with OHLC data.
        thresholds (list): List of thresholds to check for percentage change (default is [0.04, 0.07, 0.1]).
    
    Returns:
        pl.DataFrame: DataFrame with the calculated percentage of rows exceeding thresholds per year-month, 
                      including total averages.
    """
    # Calculate percentage change for high-open and open-low
    df = df.with_columns([
        ((pl.col("high") - pl.col("open")) / pl.col("open") * 100)
        .fill_nan(0)
        .alias("perc_high_open"),

        ((pl.col("open") - pl.col("low")) / pl.col("open") * 100)
        .fill_nan(0)
        .alias("perc_open_low")
    ])

    # Calculate max_abs_change as the maximum of perc_high_open and perc_open_low
    df = df.with_columns([
        pl.max_horizontal(["perc_high_open", "perc_open_low"]).alias("max_abs_change")
    ])

    # Partition the data by year_month and calculate the percentage for each threshold
    result = []
    for year_month in df["year_month"].unique():
        month_df = df.filter(pl.col("year_month") == year_month)
        total_rows = month_df.height
        
        # Calculate percentage of rows exceeding the thresholds for the current year_month
        threshold_results = {
            f"threshold_{threshold}_percent": (month_df.filter(pl.col("max_abs_change") >= threshold).height / total_rows) * 100
            for threshold in thresholds
        }
        
        # Add year_month info to the result
        threshold_results["year_month"] = year_month
        result.append(threshold_results)

    # Convert the results to a DataFrame
    result_df = pl.DataFrame(result)

    # Calculate total averages for each threshold across all months
    total_avg = {}
    for threshold in thresholds:
        total_avg[f"threshold_{threshold}_percent"] = result_df[f"threshold_{threshold}_percent"].mean()

    # Add the total averages as a new row to the DataFrame
    total_avg["year_month"] = "Total Average"
    result_df = result_df.vstack(pl.DataFrame([total_avg]))

    # Sort the result by year_month to ensure it's ordered
    result_df = result_df.sort("year_month")

    return result_df

# Example usage:
monthly_summary = calculate_max_abs_change_by_year_month(aggregated_df, thresholds=[0.04, 0.07, 0.1])

# Print the monthly summary with total average
pl.Config.set_tbl_rows(13)  # Use `set_tbl_rows` instead of `set_tbl_formatting`

# Print the monthly summary with total average
print("\nMonthly Summary of Threshold Percentages with Total Average:", monthly_summary)


Monthly Summary of Threshold Percentages with Total Average: shape: (13, 4)
┌────────────────────────┬────────────────────────┬───────────────────────┬───────────────┐
│ threshold_0.04_percent ┆ threshold_0.07_percent ┆ threshold_0.1_percent ┆ year_month    │
│ ---                    ┆ ---                    ┆ ---                   ┆ ---           │
│ f64                    ┆ f64                    ┆ f64                   ┆ str           │
╞════════════════════════╪════════════════════════╪═══════════════════════╪═══════════════╡
│ 69.52733               ┆ 40.985663              ┆ 23.266129             ┆ 2024-01       │
│ 65.505268              ┆ 36.712165              ┆ 20.82136              ┆ 2024-02       │
│ 84.034498              ┆ 59.99552               ┆ 40.456989             ┆ 2024-03       │
│ 81.981481              ┆ 55.386574              ┆ 35.405093             ┆ 2024-04       │
│ 72.352151              ┆ 43.060036              ┆ 24.912634             ┆ 2024-05       │
│ 5